In [2]:
import json
import logging
import xml.etree.ElementTree as ET
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

## Data exporation from json file for only sum of the main XML tags

In [4]:
filepath = os.path.normpath(os.getcwd() + '/..' + '/data/processed/data.json')
filepath

'/home/hazot/code/trec-clinical-trials-2023/data/processed/data.json'

In [5]:
df = pd.read_json(filepath)

In [ ]:
#TODO: try to explain each results here in text

In [6]:
print(df.shape)
print(df.columns)

(451538, 17)
Index(['nct_id', 'link_text', 'url', 'id_info', 'brief_title', 'sponsors',
       'brief_summary', 'detailed_description', 'primary_purpose',
       'intervention', 'eligibility', 'gender', 'minimum_age', 'maximum_age',
       'healthy_volunteers', 'keyword', 'condition_browse'],
      dtype='object')


In [7]:
print("Sum of column:", np.sum(list(df['gender'].value_counts())))
print('-----------')
print(df['gender'].value_counts())

Sum of column: 450667
-----------
gender
All       387617
Female     43441
Male       19609
Name: count, dtype: int64


In [8]:
print("Sum of column:", np.sum(list(df['minimum_age'].value_counts())))
print('-----------')
print(df['minimum_age'].value_counts())

Sum of column: 450667
-----------
minimum_age
18 Years     279575
N/A           30526
20 Years      17938
40 Years       9334
21 Years       9238
              ...  
84 Days           1
118 Years         1
73 Hours          1
167 Days          1
23 Days           1
Name: count, Length: 318, dtype: int64


In [48]:
# Full list of possibilities
dict(df['minimum_age'].value_counts())['N/A']

30526

In [9]:
print("Sum of column:", np.sum(list(df['maximum_age'].value_counts())))
print('-----------')
print(df['maximum_age'].value_counts())

Sum of column: 450667
-----------
maximum_age
N/A           214375
65 Years       29033
80 Years       23198
75 Years       22838
70 Years       18586
               ...  
54 Days            1
263 Months         1
93 Days            1
26 Hours           1
37 Days            1
Name: count, Length: 473, dtype: int64


In [10]:
print("Sum of column:", np.sum(list(df['healthy_volunteers'].value_counts())))
print('-----------')
print(df['healthy_volunteers'].value_counts())

Sum of column: 440292
-----------
healthy_volunteers
No                            322154
Accepts Healthy Volunteers    118138
Name: count, dtype: int64


### More complicated cases (more subtags, more intricate dicts)

In [11]:
df['intervention'][0]

[{'intervention_type': 'Biological',
  'intervention_name': 'V930',
  'description': 'V930 Over a 94 week duration, patients will receive a series of 5 injections (2.5 mg/injection), one every other week. Within 2 minutes of each injection of V930, each patient will be given an EP-IM injection consisting of two 60 msec pulses.',
  'arm_group_label': '1'},
 {'intervention_type': 'Biological',
  'intervention_name': 'V932',
  'description': 'V932 Over a 94 week duration, patients will receive a series of 5 injections, 6 patients will initially received intramuscular V932 vaccinations at a low dose (0.5x109 vg/injection),and following a safety assessment, up to an additional 35 patients will be treated with the high dose V932 (0.5x1011 vg/injection).',
  'arm_group_label': '2'}]

In [12]:
df['eligibility'][0].keys()

dict_keys(['criteria', 'gender', 'minimum_age', 'maximum_age', 'healthy_volunteers'])

In [43]:
tuple(df['eligibility'][0].items())

(('criteria',
  'Inclusion Criteria Patients must have completed surgical treatment for his/her primary disease at least 1 month prior to enrollment Patient must not be pregnant 3 days prior to enrollment Exclusion Criteria Patient is currently participating or has participated in a study with an investigational compound or device within 30 days of signing informed consent Patient has had their spleen removed or has a history of autoimmune disorders Patient is a regular user of any illicit drugs or has used within the past year of drug or alcohol abuse Patient is pregnant or breastfeeding or is expecting to conceive anytime following the study Patient is known to be Human Immunodeficiency Virus (HIV)-seropositive Patient has a known history of Hepatitis B or C Patient has received a vaccine for any disease or condition within one month of enrollment Patient has a primary central nervous system tumor.'),
 ('gender', 'All'),
 ('minimum_age', '18 Years'),
 ('maximum_age', 'N/A'),
 ('healt

In [71]:
all_possible_tags = {}

for i in tqdm(range(df.shape[0]), desc='Calculations...', disable=False, position=0, leave=True):
    if df['eligibility'][i] is None:
        continue
    for key in tuple(df['eligibility'][i].keys()):
        if key not in all_possible_tags:
            all_possible_tags[key] = 1
        else:
            all_possible_tags[key] += 1

Calculations...: 100%|███████████████████████████████████████████████████████████████████████████████████| 451538/451538 [00:02<00:00, 153367.99it/s]


In [72]:
all_possible_tags

{'criteria': 450592,
 'gender': 450667,
 'minimum_age': 450667,
 'maximum_age': 450667,
 'healthy_volunteers': 440292,
 'study_pop': 97558,
 'sampling_method': 97571,
 'gender_based': 11223,
 'gender_description': 7486}

In [73]:
all_possible_values = {}

for i in tqdm(range(df.shape[0]), desc='Calculations...', disable=False, position=0, leave=True):
    if df['eligibility'][i] is None:
        continue
    value = df['eligibility'][i].get('sampling_method')
    if value is None:
        continue
    if value not in all_possible_values:
        all_possible_values[value] = 1
    else:
        all_possible_values[value] += 1
    

Calculations...: 100%|███████████████████████████████████████████████████████████████████████████████████| 451538/451538 [00:02<00:00, 182614.13it/s]


In [74]:
all_possible_values

{'Non-Probability Sample': 70217, 'Probability Sample': 27354}

In [13]:
df['keyword'][:10]

0                  Cancers expressing HER-2 and/or CEA
1                                                 None
2    [dexmedetomidine, midazolam, remifentanil, Ele...
3                                                 None
4    [adenocarcinoma of the colon, stage I colon ca...
5                                                 None
6                                                 None
7                                                 None
8    [ADHD, ADD, Attention Deficit Hyperactivity Di...
9     [wound healing, phototoxicity, Healthy Subjects]
Name: keyword, dtype: object

In [14]:
df['condition_browse'][:10]

0                                                 None
1                                                 None
2                                                 None
3    {'mesh_term': 'Attention Deficit Disorder with...
4                   {'mesh_term': 'Colonic Neoplasms'}
5            {'mesh_term': 'Constriction, Pathologic'}
6                        {'mesh_term': 'Otitis Media'}
7                                                 None
8                                                 None
9                                                 None
Name: condition_browse, dtype: object

### Less complicated cases (less subtags)

In [75]:
df['primary_purpose'].value_counts()

primary_purpose
Treatment                          224409
Prevention                          37354
Other                               17257
Supportive Care                     17069
Basic Science                       16628
Diagnostic                          15376
Health Services Research             8414
Screening                            2988
Device Feasibility                   1075
Educational/Counseling/Training       186
Name: count, dtype: int64

In [76]:
df['nct_id'].value_counts()

nct_id
NCT00647114    1
NCT04235829    1
NCT04232995    1
NCT04239365    1
NCT04237597    1
              ..
NCT02748460    1
NCT02749500    1
NCT02741791    1
NCT02741024    1
NCT03052816    1
Name: count, Length: 451538, dtype: int64

In [77]:
count_url = 0
for i in range(len(df['url'])):
    count_url += 1 if df['url'][i] else 0
print(count_url)

451538


In [78]:
len(df['id_info'][2].keys())

2

In [79]:
max_keys = 0
max_string_keys = ''
for i in range(len(df['id_info'])):
    if max_keys >= len(df['id_info'][i].keys()):
        break
    max_keys = len(df['id_info'][i].keys())
    max_string_keys = df['id_info'][i].keys()
print(max_keys)
print(max_string_keys)

min_keys = 100
min_string_keys = ''
for i in range(len(df['id_info'])):
    if min_keys <= len(df['id_info'][i].keys()):
        break
    min_keys = len(df['id_info'][i].keys())
    min_string_keys = df['id_info'][i].keys()
print(min_keys)
print(min_string_keys)

3
dict_keys(['org_study_id', 'secondary_id', 'nct_id'])
2
dict_keys(['org_study_id', 'nct_id'])


In [80]:
df['id_info'][0].keys()

dict_keys(['org_study_id', 'secondary_id', 'nct_id'])

In [81]:
count_nct_id = 0
for i in range(len(df['id_info'])):
    count_nct_id += 1 if df['id_info'][i].get('nct_id', 0) else 0
print(count_nct_id)

451538


In [82]:
df['brief_title']

0         A Study to Test V930/V932 in Patients With Can...
1         Fed Study of Benazepril HCl and Hydrochlorothi...
2         Effects of Two Different Sedation Regimes on A...
3         Safety, Tolerability and Efficacy Study of ABT...
4         Fluorouracil and Oxaliplatin With or Without P...
                                ...                        
451533    Evaluating 18F-FDG PET/CT With Liver SUVmax-ba...
451534    Imaging of in Vivo Sigma-2 Receptor Expression...
451535    Evaluation of the Efficacy Safety and Tolerabi...
451536          Resistant Starch, Gut Bacteria and Diabetes
451537    Ice T Postoperative Multimodal Pain Regimen in...
Name: brief_title, Length: 451538, dtype: object

In [83]:
df['brief_title'].isna().sum()

0

In [84]:
df['brief_summary'].isna().sum()

841

In [85]:
df['brief_summary'].apply(type).value_counts()

brief_summary
<class 'str'>         450697
<class 'NoneType'>       841
Name: count, dtype: int64

In [86]:
df['brief_summary'].value_counts()[:10]

brief_summary
To evaluate the Sun Protection Factor efficacy on human skin.                                                                                                                                                                                                                                                                        49
Investigators are building an empirical evidence base for real world data through large-scale replication of randomized controlled trials. The investigators' goal is to understand for what types of clinical questions real world data analyses can be conducted with confidence and how to implement such studies.                35
This study aims to evaluate the comparative risk of dementia/Alzheimer's disease onset between patients treated with medications that target specific metabolic pathways and patients treated with alternative medications for the same indication.                                                                                  14
Th

In [87]:
bad_indexes = df.loc[pd.isna(df["brief_summary"]), :].index
print(bad_indexes)
print('len(bad_indexes):', len(bad_indexes))

Index([   854,    936,    977,   2962,   3354,   3359,   4953,   5235,   5307,
         5517,
       ...
       441722, 442341, 443174, 444716, 445616, 447390, 448477, 449224, 449228,
       449655],
      dtype='int64', length=841)
len(bad_indexes): 841


### Check word (tokens) frequency

1. Word frequency in all strings

In [88]:
df['brief_summary'][:10000].str.split(expand=True).stack().value_counts()[:30]

the          37181
of           35599
and          25735
to           24576
in           20003
a            13574
is           13541
with         12917
will          9869
study         9650
The           8380
for           8205
or            7030
be            7026
patients      6955
this          6022
that          4916
are           4429
This          4273
as            4032
by            3847
treatment     3830
on            3829
may           3696
have          3683
an            2986
at            2926
who           2866
from          2823
blood         2718
Name: count, dtype: int64

In [89]:
df['brief_summary']

0         Treatment of patients with cancer types known ...
1         The objective of this study was to investigate...
2         Sedation may be necessary in intensive care to...
3         The purpose of this study is to test if the in...
4         RATIONALE Drugs used in chemotherapy, such as ...
                                ...                        
451533    The purpose of this study is to evaluate wheth...
451534    Pilot study using [18F]ISO-1 PET/CT to image s...
451535    Double blind, randomized multi-center, evaluat...
451536    The aim of the study is to investigate, if res...
451537    The purpose of this randomized controlled tria...
Name: brief_summary, Length: 451538, dtype: object

2. Token frequency in all strings

In [90]:
tokenizer = None

## Exploration of the XML tags

In [ ]:
def get_file_names(raw_data_folder_path):
    file_paths = []
    for splits in os.scandir(raw_data_folder_path):
        if splits.is_dir():
            for nct_dir in os.scandir(splits.path):
                for file in os.scandir(nct_dir.path):
                    if file.is_file():
                        file_paths.append(file.path)
    return file_paths

In [ ]:
raw_data_folder_path = os.path.normpath(os.getcwd() + '/..' + '/data/raw/')
raw_data_folder_path

In [ ]:
def tag_exploration_loop_raw_numbers(raw_data_folder_path):
    file_paths = get_file_names(raw_data_folder_path)
    
    nb_elems_all = []
    max_elem_list = []
    all_tags_list = set()
    for file_path in tqdm(file_paths, desc='Parsing XML files to json:', disable=False, position=0, leave=True):
        # load and parse the file
        xml_tree = ET.parse(file_path)
        
        elem_list = []
        
        for elem in xml_tree.iter():
            elem_list.append(elem.tag)
        
        # now I remove duplicities - by convertion to set and back to list
        elem_list = list(set(elem_list))
    
        nb_elem = len(elem_list)
        nb_elems_all.append(nb_elem)
        
        if nb_elem > len(max_elem_list):
            max_elem_list = elem_list

        for tag in elem_list:
            if tag not in all_tags_list:
                all_tags_list.add(tag)

    return nb_elems_all, max_elem_list, all_tags_list

In [ ]:
result_tuple = tag_exploration_loop_raw_numbers(raw_data_folder_path)

In [ ]:
list_nb_of_elems = result_tuple[0]
list_max_elem = result_tuple[1]
all_possible_tags_list = result_tuple[2]

In [ ]:
# Statistics on the number of elems in list
import statistics as stats
print('Number of Clinical Trials possible:', len(list_nb_of_elems))
print('Average number of tags in a list:', np.mean(list_nb_of_elems))
print('std of number of tags in a list:', np.std(list_nb_of_elems))

In [ ]:
# analysis of a list with the most elements in it
print('Number of tags in the XML file with the most tags:', len(list_max_elem))
print('Number of possible tags in all files:', len(all_possible_tags_list))

In [ ]:
# analysis of a list of all possible tags
print(all_possible_tags_list)